In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from gomat_markup_opt.data.preparation import (
    load_and_prepare_inventory_data,
    load_and_prepare_quote_lost_data,
    load_and_prepare_quote_order_data,
)
from gomat_markup_opt.models.conversion_probability_model import ConversionProbabilityModel
from gomat_markup_opt.preprocessing.cleanup import (
    cleanup_and_filter_inventory_data,
    cleanup_and_filter_quote_and_order_data,
)
from gomat_markup_opt.preprocessing.feature_engineering import engineer_features

TARGET_COLUMN = "target"

data_dir = os.path.abspath(os.path.join('..', 'data'))
if os.path.exists(data_dir):
    os.environ['DATA_DIR'] = data_dir
    print(f"Set DATA_DIR to: {data_dir}")
else:
    print(f"Warning: Data directory not found at {data_dir}. Ensure DATA_DIR is set correctly.")

prepared_quote_order_data = load_and_prepare_quote_order_data()
prepared_quote_lost_data = load_and_prepare_quote_lost_data()
prepared_inventory_data = load_and_prepare_inventory_data()

cleaned_quote_data = cleanup_and_filter_quote_and_order_data(prepared_quote_order_data, prepared_quote_lost_data)
cleaned_inventory_data = cleanup_and_filter_inventory_data(prepared_inventory_data)

engineered_data = engineer_features(cleaned_quote_data, cleaned_inventory_data, TARGET_COLUMN)

engineered_data.head()

In [ ]:
numerical_columns = [
    "quantity",
    "seller_price",
    "buyer_price",
    "seller_total",
    "markup",
]

categorical_columns = [
    "type",
    "qto",
    "buyer_region",
    "plant_category",
    "delivery_method",
    "size",
]

filtered_data = engineered_data[numerical_columns + categorical_columns]
filtered_data = filtered_data.dropna()


In [ ]:

corr = filtered_data[numerical_columns].corr()["markup"].drop("markup")
plt.figure(figsize=(10, 6))
plt.title("Correlation with Markup")
sns.barplot(x=corr.index, y=corr.values)
plt.xlabel("Features")
plt.ylabel("Correlation Coefficient")
plt.show()


In [ ]:
for col in filtered_data.columns:
    if col == "markup" or col =="target":
        continue

    # violin plot for categorical features
    if col in categorical_columns:
        plt.figure(figsize=(10, 6))
        sns.violinplot(x=col, y="markup", data=filtered_data)
        plt.title(f"Markup vs {col}")
        plt.xlabel(col)
        plt.ylabel("Markup")
        plt.show()
        continue
    
    # scatter plot for numerical features
    plt.figure(figsize=(10, 6))
    plt.scatter(filtered_data[col], filtered_data["markup"], alpha=0.5)
    plt.title(f"Markup vs {col}")
    plt.xlabel(col)
    plt.ylabel("Markup")
    plt.show()